# Use MatMiner's tools to get descriptors, and fit machine learning models to ~10,000 calculated band gap from Materials Project

This notebook is an example of using the MP data retrieval tool **'retrieve_MP.py'** to retrieve computed band gaps from MP's databases (https://www.materialsproject.org/) in the form of a Pandas dataframe, using MatMiner's tools to populate the dataframe with descriptors/features from pymatgen, and then fitting regression models from the **scikit-learn** library to the dataset.

## Preamble

###  Import libraries, and set pandas options to display all rows and columns

In [1]:
# filter warnings messages from the notebook
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# Set pandas view options
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Step 1: Use MatMiner to (i) obtain calculated band gap and other data from MP and (ii) (automatically) put that data in a "pandas" dataframe

### Step 1a:  Import MatMiner's MP data retrieval tool and get the data

In [2]:
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval

api_key = None   # Set your MP API key here. If set as an environment variable 'MAPI_KEY', set it to 'None'
mpr = MPDataRetrieval(api_key)     # Create an adapter to the MP Database.

# criteria is to get all binary compounds
criteria = {'nelements': 2}

# properties are the materials attributes we want
# See https://github.com/materialsproject/mapidoc for available properties you can specify
properties = ['pretty_formula', 'spacegroup.symbol', 'formation_energy_per_atom', 'band_gap', 'e_above_hull', 
              'density', 'volume', 'nsites']

# get the data!
df_mp = mpr.get_dataframe(criteria=criteria, properties=properties)
print 'Number of binary compounds extracted = {}'.format(len(df_mp))

Number of binary compounds extracted = 10338


### Step 1b: Explore the dataset

In [3]:
df_mp.head()

,pretty_formula,spacegroup.symbol,formation_energy_per_atom,band_gap,e_above_hull,density,volume,nsites
material_id,,,,,,,,
mp-656887,Ni3O4,Cmmm,-0.801768,0.3815,0.022152,5.547050,71.868570,7
mp-1703,YbZn,Pm-3m,-0.330870,0.0000,0.000000,8.403950,47.115209,2
mp-569624,HfCr2,P6_3/mmc,-0.096054,0.0000,0.017904,10.495619,178.768942,12
mp-12660,LuB6,Pm-3m,-0.309740,0.0000,0.089608,5.881854,67.708585,7
mp-188,AlPt3,Pm-3m,-0.685692,0.0000,0.007060,16.827942,60.413663,4


In [4]:
df_mp.describe()

,formation_energy_per_atom,band_gap,e_above_hull,density,volume,nsites
count,10338.000000,10338.000000,10338.000000,10338.000000,10338.000000,10338.000000
mean,-0.758933,0.664137,0.109162,6.796724,349.202493,16.471271
std,1.017048,1.404584,0.257421,3.637525,541.455442,23.171972
min,-4.522499,0.000000,0.000000,0.212291,11.917209,2.000000
25%,-1.118783,0.000000,0.000000,4.065629,80.374364,4.000000
50%,-0.438731,0.000000,0.006421,6.332125,162.685652,8.000000
75%,-0.129333,0.501650,0.089608,8.704261,374.100130,19.000000
max,2.410713,9.061200,4.244913,21.676368,7697.812543,200.000000


### Step 1c. Filter out unstable entries

In [5]:
df_mp = df_mp[df_mp['e_above_hull'] < 0.1]
df_mp.describe()

,formation_energy_per_atom,band_gap,e_above_hull,density,volume,nsites
count,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000,7892.000000
mean,-0.852159,0.737277,0.013288,6.829394,385.658210,17.823999
std,0.950654,1.471430,0.023634,3.551834,567.481828,23.991900
min,-4.522499,0.000000,0.000000,0.212291,11.917209,2.000000
25%,-1.117593,0.000000,0.000000,4.151389,91.659544,4.000000
50%,-0.487314,0.000000,0.000000,6.418981,184.168242,8.000000
75%,-0.218557,0.756025,0.016032,8.738857,420.946231,20.000000
max,0.099855,9.061200,0.099855,21.676368,6897.599713,196.000000


## Step 2: Add descriptors/features

### Step 2a: create volume per atom descriptor

In [6]:
# add volume per atom descriptor
df_mp['vpa'] = df_mp['volume']/df_mp['nsites']

# explore columns
df_mp.head()

,pretty_formula,spacegroup.symbol,formation_energy_per_atom,band_gap,e_above_hull,density,volume,nsites,vpa
material_id,,,,,,,,,
mp-656887,Ni3O4,Cmmm,-0.801768,0.3815,0.022152,5.547050,71.868570,7,10.266939
mp-1703,YbZn,Pm-3m,-0.330870,0.0000,0.000000,8.403950,47.115209,2,23.557604
mp-569624,HfCr2,P6_3/mmc,-0.096054,0.0000,0.017904,10.495619,178.768942,12,14.897412
mp-12660,LuB6,Pm-3m,-0.309740,0.0000,0.089608,5.881854,67.708585,7,9.672655
mp-188,AlPt3,Pm-3m,-0.685692,0.0000,0.007060,16.827942,60.413663,4,15.103416


### Step 2b: add several more descriptors using MatMiner's pymatgen descriptor getter tools

In [7]:
from matminer.descriptors.composition_features import get_pymatgen_descriptor
from pymatgen import Composition

descriptors = ['row', 'group', 'atomic_mass', 
               'atomic_radius', 'boiling_point', 'melting_point', 'X']

for d in descriptors:
    df_mp[d] = df_mp['pretty_formula'].map(lambda x: np.mean(get_pymatgen_descriptor(Composition(x), d)))

df_mp.head()

,pretty_formula,spacegroup.symbol,formation_energy_per_atom,band_gap,e_above_hull,density,volume,nsites,vpa,row,group,atomic_mass,atomic_radius,boiling_point,melting_point,X
material_id,,,,,,,,,,,,,,,,
mp-656887,Ni3O4,Cmmm,-0.801768,0.3815,0.022152,5.547050,71.868570,7,10.266939,2.857143,13.428571,34.296829,0.921429,1416.971429,771.885714,2.784286
mp-1703,YbZn,Pm-3m,-0.330870,0.0000,0.000000,8.403950,47.115209,2,23.557604,6.000000,14.000000,119.224500,1.550000,1324.500000,894.840000,0.825000
mp-569624,HfCr2,P6_3/mmc,-0.096054,0.0000,0.017904,10.495619,178.768942,12,14.897412,4.666667,5.333333,94.160733,1.450000,3588.000000,2288.666667,1.540000
mp-12660,LuB6,Pm-3m,-0.309740,0.0000,0.089608,5.881854,67.708585,7,9.672655,2.857143,13.571429,34.261857,0.978571,4125.000000,2288.428571,1.930000
mp-188,AlPt3,Pm-3m,-0.685692,0.0000,0.007060,16.827942,60.413663,4,15.103416,5.250000,10.750000,153.058385,1.325000,3771.500000,1764.417500,2.112500


## Step 3: Fit a Linear Regression model, get R<sup>2</sup> and RMSE

### Step 3a: Define what column is the target output, and what are the relevant descriptors

In [8]:
# target output column
y = df_mp['band_gap'].values

# possible descriptor columns
X_cols = [c for c in df_mp.columns 
          if c not in ['band_gap', 'pretty_formula', 
                       'volume', 'nsites', 'spacegroup.symbol', 'e_above_hull']]
X = df_mp.as_matrix(X_cols)

print("Possible descriptors are: {}".format(X_cols))


Possible descriptors are: ['formation_energy_per_atom', 'density', 'vpa', 'row', 'group', 'atomic_mass', 'atomic_radius', 'boiling_point', 'melting_point', 'X']


### Step 3b: Fit the linear regression model

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_regression = LinearRegression()

linear_regression.fit(X, y)

# get fit statistics
print 'R2 = ' + str(round(linear_regression.score(X, y), 3))
print 'RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=linear_regression.predict(X)))

R2 = 0.552
RMSE = 0.985


### Step 3c: Plot the results using FigRecipes

In [10]:
from figrecipes.plotly.make_plots import PlotlyFig

PlotlyFig(x_title='DFT (MP) band gap (eV)', y_title='Linear model band gap (eV)', plot_title='Linear regression', 
       plot_mode='notebook', margin_left=150, textsize=35, ticksize=30)\
    .xy_plot(x_col=y, y_col=linear_regression.predict(X), marker_outline_width=1, 
                 text=df_mp['pretty_formula'], add_xy_plot=[{'x_col': [0, 9], 'y_col': [0, 9],
                                                                'color': 'black', 'mode': 'lines', 'legend': None,
                                                                'text': None, 'size': None}])

### Step 3d: Cross validate the results

In [11]:
from sklearn.cross_validation import KFold, cross_val_score

# Use 10-fold cross validation (90% training, 10% test)
crossvalidation = KFold(n=X.shape[0], n_folds=10, shuffle=True, random_state=1)

# compute cross validation scores for random forest model
scores = cross_val_score(linear_regression, X, y, scoring='mean_squared_error', 
                         cv=crossvalidation, n_jobs=1)
rmse_scores = [np.sqrt(abs(s)) for s in scores]

print 'Cross-validation results:'
print 'Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores)))

Cross-validation results:
Folds: 10, mean RMSE: 0.987


### Follow similar steps for a random forest regression model

### Step 3b: Fit the random forest model

In [12]:
from sklearn.ensemble import RandomForestRegressor

RF_rg = RandomForestRegressor(n_estimators=50, random_state=1)

RF_rg.fit(X, y)
print 'R2 = ' + str(round(RF_rg.score(X, y), 3))
print 'RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=RF_rg.predict(X)))

R2 = 0.987
RMSE = 0.168


### Step 3c: Plot the results using FigRecipes

In [13]:
from figrecipes.plotly.make_plots import PlotlyFig

PlotlyFig(x_title='DFT (MP) band gap (eV)', y_title='Random forest band gap (eV)', plot_title='Random forest regression',
       plot_mode='notebook', margin_left=150, textsize=35, ticksize=30)\
    .xy_plot(x_col=y, y_col=RF_rg.predict(X), marker_outline_width=1, 
                 text=df_mp['pretty_formula'], add_xy_plot=[{'x_col': [0, 9], 'y_col': [0, 9],
                                                                'color': 'black', 'mode': 'lines', 'legend': None,
                                                                'text': None, 'size': None}])

### Step 3d: Cross validate the results

In [14]:
# compute cross validation scores for random forest model
scores = cross_val_score(RF_rg, X, y, scoring='mean_squared_error', 
                         cv=crossvalidation, n_jobs=1)

rmse_scores = [np.sqrt(abs(s)) for s in scores]

print 'Cross-validation results:'
print 'Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores)))

Cross-validation results:
Folds: 10, mean RMSE: 0.440
